In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
ue_i = pd.read_excel('./data/data_raw/unemployment_insurance.xls')
pop = pd.read_csv('./data/pop_df_merged.csv')

In [3]:
ue_i.columns = ue_i.columns = [x.lower().replace(' ', '_') for x in list(ue_i[3:4].values.flatten())]

In [5]:
#removing excel formatting issues
ue_i = ue_i.drop(labels = list(range(0,4)), axis = 0).reset_index(drop = True)
ue_i = ue_i.dropna()

In [6]:
def by_state(dataframe):
    df_ic = pd.DataFrame()
    df_cc = pd.DataFrame()
    df_iur = pd.DataFrame()
    df_ce = pd.DataFrame()

    dfs = [df_ic, df_cc, df_iur, df_ce]

    #each state is a new column
    for x in dataframe['state'].unique():
        df1 = dataframe[dataframe['state'] == x]
        df_ic[x] = df1['initial_claims'].reset_index(drop = True)
        df_cc[x] = df1['continued_claims'].reset_index(drop = True)
        df_iur[x] = df1['insured_unemployment_rate'].reset_index(drop = True)
        df_ce[x] = df1['covered_employment'].reset_index(drop = True)

    # setting time-series column\n",
    for df in dfs:
        df['reflecting_week_ended'] = dataframe['reflecting_week_ended'][:545].astype('datetime64[ns]')

    return df_ic, df_cc, df_ce, df_iur

In [7]:
ic, cc, ce, iur = by_state(ue_i)

In [9]:
def index_monthly(df):
    df = df.set_index('reflecting_week_ended')
    df = df.resample('M').mean().ffill()
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    mask = (df.index > '2012') & (df.index < '2022')
    df = df[mask]
    df = df.drop(columns = ['virgin_islands'])
    return df

In [10]:
ic = index_monthly(ic)
cc = index_monthly(cc)
ce = index_monthly(ce)
iur = index_monthly(iur)

### Aligning pop and ue_i datasets to create per cap unemployment insurance

In [11]:
# removing ['2010-04-01','2020-04-01']\n",
pop1 = pop.copy()
pop1 = pop1.drop(labels= [0,11], axis = 0)

In [12]:
pop1['Unnamed: 0'] = pop1['Unnamed: 0'].map(lambda x: x.replace('07', '12'))

In [13]:
def mod_pop(df):
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    df['unnamed:_0'] = pd.to_datetime(df['unnamed:_0'])
    df = df.set_index('unnamed:_0')
    df = df.resample('M').ffill()
    start = (df.index > '2012')
    df = df[start]
    return df

In [14]:
pop1 = mod_pop(pop1)

In [15]:
def per_cap(df):
    df = df.div(pop1)
    return df

In [16]:
ic = per_cap(ic)
cc = per_cap(cc)
ce = per_cap(ce)
iur = per_cap(iur)

In [ ]:
#time series- 'filed_week_ended'\n",
ic.to_csv('./data/percap_initial_claims.csv')
cc.to_csv('./data/percap_continued_claims.csv')
ce.to_csv('./data/percap_covered_employment.csv')
iur.to_csv('./data/percap_insured_unemployment_rate.csv')